In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sqlite3
import sys
import logging

# Data
from sklearn.model_selection import train_test_split

# Tensorflow
import tensorflow as tf

# Graph
import tensorflow_gnn as tfgnn

# Add the project root to the Python path
#sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from scripts.data_cleaner import filter_top_cpv_categories
from scripts.gnn_anomaly_detection import ProcurementGraphBuilder, GNNAnomalyDetector, AnomalyAnalyzer


%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
graph_builder = ProcurementGraphBuilder()

data_path = os.path.join(os.path.dirname(os.getcwd()), 'data')

df = graph_builder.load_data(data_path)

X_train_preproc, X_test_preproc = graph_builder.preprocess_data(df)

X_train_preproc.head()

INFO:scripts.gnn_anomaly_detection:Loading data from /home/ronan/code/RonanB400/Project/decp_ml/data
INFO:scripts.gnn_anomaly_detection:Preprocessing data...


Filtered from 392 to 80 CPV categories, keeping 261824 rows out of 286850


/home/ronan/.pyenv/versions/3.10.6/envs/decp_ml_env/lib/python3.10/site-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


,offres_recues_pipeline__offresRecues,other_num_pipeline__montant,other_num_pipeline__dureeMois,binary_pipeline__sousTraitanceDeclaree,binary_pipeline__origineFrance,binary_pipeline__marcheInnovant,binary_pipeline__idAccordCadre,cat_pipeline__procedure_Appel d'offres ouvert,cat_pipeline__procedure_Appel d'offres restreint,cat_pipeline__procedure_Dialogue compétitif,...,cat_pipeline__codeCPV_3_90600000,cat_pipeline__codeCPV_3_90700000,cat_pipeline__codeCPV_3_90900000,cat_pipeline__codeCPV_3_9100000,cat_pipeline__codeCPV_3_92300000,cat_pipeline__codeCPV_3_92500000,cat_pipeline__codeCPV_3_9300000,cat_pipeline__codeCPV_3_98300000,acheteur_nom,titulaire_nom
0,-0.589339,-0.448980,-2.481278,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,COMMUNE DE SAINT ETIENNE DE TINEE,SERMATECH
1,-1.350105,-0.829173,0.642652,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,COMMUNE DE CHALMOUX,MACON ETANCHEITE
2,1.669651,-0.216880,0.222909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CC DES PORTES D'ARIEGE PYRENEES,EQUADEX
3,-0.049567,0.912515,0.943402,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4,0.369113,-0.425268,-0.016001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,COMMUNE DE SAINT SAULVE,ALTOMARE ALTALU


In [9]:
X_train_preproc.columns.tolist()

['offres_recues_pipeline__offresRecues',
 'other_num_pipeline__montant',
 'other_num_pipeline__dureeMois',
 'binary_pipeline__sousTraitanceDeclaree',
 'binary_pipeline__origineFrance',
 'binary_pipeline__marcheInnovant',
 'binary_pipeline__idAccordCadre',
 "cat_pipeline__procedure_Appel d'offres ouvert",
 "cat_pipeline__procedure_Appel d'offres restreint",
 'cat_pipeline__procedure_Dialogue compétitif',
 'cat_pipeline__procedure_Marché passé sans publicité ni mise en concurrence préalable',
 'cat_pipeline__procedure_Procédure adaptée',
 'cat_pipeline__procedure_Procédure avec négociation',
 'cat_pipeline__nature_Marché',
 'cat_pipeline__nature_Marché de défense ou de sécurité',
 'cat_pipeline__nature_Marché de partenariat',
 'cat_pipeline__formePrix_Forfaitaire',
 'cat_pipeline__formePrix_Mixte',
 'cat_pipeline__formePrix_Unitaire',
 'cat_pipeline__ccag_Fournitures courantes et services',
 "cat_pipeline__ccag_Maitrise d'œuvre",
 'cat_pipeline__ccag_Marchés industriels',
 'cat_pipeline_

In [5]:
X_train_graph = graph_builder.create_graph(X_train_preproc)

INFO:scripts.gnn_anomaly_detection:Creating graph structure...


KeyError: 'acheteur_nom'